In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import pickle
import holidays
from sklearn.preprocessing import LabelEncoder

with open('/content/drive/MyDrive/Colab Notebooks/GBDS_Project/raw_data/station_encoder.pkl', 'rb') as handle:
    le = pickle.load(handle)

In [3]:
import holidays
us_holidays = holidays.country_holidays('US')

In [4]:
standard_stations = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/standard_stations.csv").drop(columns=['Unnamed: 0'])
weather_data_time = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/weather_data.csv").drop(columns=['Unnamed: 0'])
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/df_final_dataframe_grouped.csv").drop(columns=['Unnamed: 0'])

In [5]:
standard_stations['station_id'] = le.transform(standard_stations['station_id'])

In [6]:
weather_data_time['started_at_hourly'] = pd.to_datetime(weather_data_time['started_at_hourly'])

In [7]:
all_clusters = np.union1d(
    df['start_station_cluster_centroid'].unique(),
    df['end_station_cluster_centroid'].unique()
  )

In [8]:
centroid_label_encoder = LabelEncoder()

In [9]:
all_clusters = centroid_label_encoder.fit_transform(all_clusters)



with open('/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/centroid_encoder.pkl', 'wb') as handle:
    pickle.dump(centroid_label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
#conversion_dict_hourly = dict(year= df[f'started_at_year'],
#                              month=df[f'started_at_month'],
#                              day=  df[f'started_at_day'],
#                              hour= df[f'started_at_hour']
#                              )
#df[f'started_at_hourly'] = pd.to_datetime(conversion_dict_hourly)
#
#
#df = df[df['started_at_hourly'] >  pd.to_datetime("2024-01-31 11:00:00")]

In [10]:
graphs = []
features = []
count = 0
file_count = 0
processed_data_loc = '/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/'
for (year, month, day, hour, quarter), group in df.groupby([
      'started_at_year',
      'started_at_month',
      'started_at_day',
      'started_at_hour',
      'started_at_quarter',
    ]):

    graph_matrix = np.zeros((len(all_clusters), len(all_clusters)))
    duration_matrix = np.zeros((len(all_clusters), len(all_clusters)))

    group['start_station_cluster_centroid'] = centroid_label_encoder.transform(group['start_station_cluster_centroid'])
    group['end_station_cluster_centroid'] = centroid_label_encoder.transform(group['end_station_cluster_centroid'])
    for _, row in group.iterrows():
        pickup_idx = np.where(all_clusters == row['start_station_cluster_centroid'])[0][0]
        dropoff_idx = np.where(all_clusters == row['end_station_cluster_centroid'])[0][0]
        graph_matrix[pickup_idx, dropoff_idx] = row['demand']

        duration_matrix[pickup_idx, dropoff_idx] = row['duration_sec']

    features_list = []
    for i in range(len(graph_matrix)):
        temp_features = []
        temp_features.append(year)
        temp_features.append(quarter)
        temp_features.append(month)
        temp_features.append(day)
        temp_features.append(hour)
        temp_features.append(graph_matrix[i].sum())

        query = f"""
          started_at_year== {year} &\
          started_at_month == {month} &\
          started_at_day == {day} &\
          started_at_hour == {hour} &\
          start_station_cluster_centroid == {i}
        """

        weather_features = weather_data_time[
            (weather_data_time['started_at_hourly'] == pd.to_datetime(f"{year}-{month}-{day} {hour}:00:00")) &
            (weather_data_time['centroid'] == 1)
        ][['temp', 'wspd', 'rhum']]
        for feat in weather_features:
          temp_features.append(weather_features[feat].iloc[0])
        duration = duration_matrix[i][duration_matrix[i] != 0]
        if duration.shape[0]!=0:
          temp_features.append(duration_matrix[i][duration_matrix[i] != 0].mean())
        else:
          temp_features.append(0)
        features_list.append(temp_features)

    features.append(features_list)
    graphs.append(graph_matrix)
    count+=1
    chunk_size = 1000
    if count%chunk_size == 0:

      print(count)
      print(pd.to_datetime(f"{year}-{month}-{day} {hour}:00:00"))
      num_chunks = len(features) // chunk_size

      np.save(f'{processed_data_loc}features/features_156_clusters_2024_4_11__{file_count}.npy', features[(file_count) * chunk_size: (file_count + 1) * chunk_size])
      np.save(f'{processed_data_loc}graphs/graphs_156_clusters_2024_4_11__{file_count}.npy', graphs[(file_count) * chunk_size: (file_count + 1) * chunk_size])

      file_count+=1

NameError: name 'ss' is not defined

In [13]:
graph_matrix.shape

(156, 156)

In [21]:
features[0]

[[2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 2.0, 15.0, 16.6, 67.0, 393.0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 1.0, 15.0, 16.6, 67.0, 193.0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1, 31, 12, 0.0, 15.0, 16.6, 67.0, 0],
 [2024, 1, 1

In [23]:
graphs[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [14]:
file_count

28

In [19]:
#file_count+=1
np.save(f'{processed_data_loc}features/features_156_clusters_2024_4_11__{file_count}.npy', features[file_count-28 * chunk_size:])
np.save(f'{processed_data_loc}graphs/graphs_156_clusters_2024_4_11__{file_count}.npy', graphs[file_count-28 * chunk_size:])


In [ ]:
file_count+=1
np.save(f'{processed_data_loc}features/features_156_clusters_2024_4_11__{file_count}.npy', features[file_count * chunk_size:])
np.save(f'{processed_data_loc}graphs/graphs_156_clusters_2024_4_11__{file_count}.npy', graphs[file_count * chunk_size:])


In [ ]:
loaded_array = np.load('/content/drive/MyDrive/Colab Notebooks/GBDS_Project/processed_data/graphs/graphs_156_clusters_2024_4_11__0.npy')
